In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv("tweets2020.csv")

In [3]:
df.head()

,2
0,RT @AjudemSerena: IMPLORO POR SOCORRO!\nPor fa...
1,@ShakedownVail Yo. What a #kickass #chill bar ...
2,"RT @grazianopascale: ""Parece la trama de un fi..."
3,RT @marcofeliciano: Pare e compare: PETROBRAS ...
4,RT @NetoItirussu: Hoje em depoimento da operaç...


In [4]:
df['Classificação'] = pd.Series(dtype=int)

In [5]:
df.head()

,2,Classificação
0,RT @AjudemSerena: IMPLORO POR SOCORRO!\nPor fa...,NaN
1,@ShakedownVail Yo. What a #kickass #chill bar ...,NaN
2,"RT @grazianopascale: ""Parece la trama de un fi...",NaN
3,RT @marcofeliciano: Pare e compare: PETROBRAS ...,NaN
4,RT @NetoItirussu: Hoje em depoimento da operaç...,NaN
